# Using RoBERTa with Fastai - ReCoRD

In [1]:
from fastai.text import *
from fastai.metrics import *
from transformers import RobertaTokenizer
import jsonlines

In [2]:
# Creating a config class to store task specific information
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

In [3]:
# defining our configuration

config = Config(
    task = "ReCoRD",
#     remove_percent = None, # Set to a num between 0 and 1 to remove num% from training ds (for quicker training)
    remove_percent = .99, # Set to None if not testing other this percetage will be removed from training ds
    undersample = True, # will fix the class imbalance by undersampling
    seq_len_check = False, # will remove all inputs over the max seq length (as opposed to truncating them)
    seed = 2019,
    roberta_model_name='roberta-base', # can also be exchanged with roberta-large 
    pad_idx = 0,
    max_lr=1e-5,
    epochs=2,
    use_fp16=False,
    bs=4, 
    max_seq_len=256, 
    num_labels = 2,
    hidden_dropout_prob=.05,
    hidden_size=768, # 1024 for roberta-large
    start_tok = "<s>",
    end_tok = "</s>",
    mask_tok = "<mask>",
    mark_fields=True,
)

In [4]:
path = Path(".")
data_path = path/"data"

In [5]:
#load json object
def record_parser(path_to_json, test=False):
    with jsonlines.open(path_to_json) as f:
        idx,passage_idx,passage,question,entity,label = [],[],[],[],[],[]
        for obj in f:
            text = obj["passage"]["text"]
            for q in obj['qas']:
                if not test: answers = [a["text"] for a in q["answers"]] 
                for e in obj["passage"]["entities"]:
                    idx.append(q["idx"])
                    passage_idx.append(obj["idx"])
                    passage.append(text)
                    ques = q["query"]#.replace("@placeholder",config.mask_tok)
                    question.append(ques)
                    ent = text[e["start"]:e["end"]+1]
                    entity.append(ent)
                    label.append(ent in answers) if not test else label.append(None)
    return pd.DataFrame({"idx":idx,"passage_idx":passage_idx,"passage":passage,"question":question,"entity":entity,"label":label})

In [6]:
train = record_parser(data_path/config.task/"train.jsonl")
val = record_parser(data_path/config.task/"val.jsonl")
test = record_parser(data_path/config.task/"test.jsonl",test=True)

In [8]:
train.tail()

,idx,passage_idx,passage,question,entity,label
1796851,100729,65708,By Kieran Gill Anton Ferdinand has signed for ...,Fresh start: Ferdinand will hope he gets more ...,Thai,False
1796852,100729,65708,By Kieran Gill Anton Ferdinand has signed for ...,Fresh start: Ferdinand will hope he gets more ...,Ferdinand,False
1796853,100729,65708,By Kieran Gill Anton Ferdinand has signed for ...,Fresh start: Ferdinand will hope he gets more ...,Antalyaspor,False
1796854,100729,65708,By Kieran Gill Anton Ferdinand has signed for ...,Fresh start: Ferdinand will hope he gets more ...,Bursaspor,False
1796855,100729,65708,By Kieran Gill Anton Ferdinand has signed for ...,Fresh start: Ferdinand will hope he gets more ...,Turkey,True


In [8]:
train.passage[0]

"The harrowing stories of women and children locked up for so-called 'moral crimes' in Afghanistan's notorious female prison have been revealed after cameras were allowed inside. Mariam has been in Badam Bagh prison for three months after she shot a man who just raped her at gunpoint and then turned the weapon on herself - but she has yet to been charged. Nuria has eight months left to serve of her sentence for trying to divorce her husband. She gave birth in prison to her son and they share a cell together. Scroll down for video Nuria was jailed for trying to divorce her husband. Her son is one of 62 children living at Badam Bagh prison\n@highlight\nMost of the 202 Badam Bagh inmates are jailed for so-called 'moral crimes'\n@highlight\nCrimes include leaving their husbands or refusing an arrange marriage\n@highlight\n62 children live there and share cells with their mothers and five others"

In [9]:
train.question[0]

'The baby she gave birth to is her husbands and he has even offered to have the courts set her free if she returns, but @placeholder has refused.'

In [10]:
train.label.value_counts()

False    1537301
True      259555
Name: label, dtype: int64

In [11]:
feat_cols = ["question","passage","entity"]
label_cols = "label"

## Setting Up the Tokenizer

In [12]:
class FastAiRobertaTokenizer(BaseTokenizer):
    """Wrapper around RobertaTokenizer to be compatible with fastai"""
    def __init__(self, tokenizer: RobertaTokenizer, max_seq_len: int=128, **kwargs): 
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len 
    def __call__(self, *args, **kwargs): 
        return self 
    def tokenizer(self, t:str) -> List[str]: 
        """Adds Roberta bos and eos tokens and limits the maximum sequence length""" 
        if config.mark_fields:
            sub = 2 # subtraction in totoal seq_length to be made due to adding spcl tokens
            assert "xxfld" in t
            t = t.replace("xxfld 1","") # remove the xxfld 1 special token from fastai
            # converting fastai field sep token to Roberta
            t = re.split(r'xxfld \d+', t) 
            res = []
            for i in range(len(t)-1): # loop over the number of additional fields and the Roberta sep
                res += self._pretrained_tokenizer.tokenize(t[i]) + [config.end_tok]
                sub += 1 # increase our subtractions since we added more spcl tokens
            res += self._pretrained_tokenizer.tokenize(t[-1]) # add the last sequence
            return [config.start_tok] + res[:self.max_seq_len - sub] + [config.end_tok] 
        
        res = self._pretrained_tokenizer.tokenize(t)
        return [config.start_tok] + res[:self.max_seq_len - sub] + [config.end_tok]

In [13]:
# create fastai tokenizer for roberta
roberta_tok = RobertaTokenizer.from_pretrained("roberta-base")

fastai_tokenizer = Tokenizer(tok_func=FastAiRobertaTokenizer(roberta_tok, max_seq_len=config.max_seq_len), 
                             pre_rules=[], post_rules=[])

In [14]:
# create fastai vocabulary for roberta
roberta_tok.save_vocabulary(path)

with open('vocab.json', 'r') as f:
    roberta_vocab_dict = json.load(f)
    
fastai_roberta_vocab = Vocab(list(roberta_vocab_dict.keys()))

In [15]:
# Setting up pre-processors
class RobertaTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
         super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False, mark_fields=config.mark_fields)

class RobertaNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=fastai_roberta_vocab, **kwargs)


def get_roberta_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for Roberta
    We remove sos and eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original Roberta model.
    """
    return [RobertaTokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor(vocab=vocab)]

## Accounting for DF Size and Sequence Length

In [16]:
from imblearn.under_sampling import RandomUnderSampler
def undersample(df):
    rus = RandomUnderSampler(random_state=config.seed)
    X_train, y_train = rus.fit_resample(df[feat_cols], df[label_cols])
    res = pd.DataFrame(X_train)
    res["label"] = y_train
    res.columns = df[["question","passage","entity"]+[label_cols]].columns
    return res

In [17]:
if config.undersample: train = undersample(train)

In [18]:
# reduce df sizes if testing

np.random.seed(config.seed)
    
if config.remove_percent: # if testing
    print(int(train.shape[0]*config.remove_percent), "of the training dataset is removed")
    
    remove_percent = config.remove_percent
    train_drop_indices = np.random.choice(train.index, int(train.shape[0]*remove_percent), replace=False)
    train.drop(train_drop_indices, inplace=True)
    
    val_drop_indices = np.random.choice(val.index, int(val.shape[0]*remove_percent), replace=False)
    val.drop(val_drop_indices, inplace=True)
    
    test_drop_indices = np.random.choice(test.index, int(test.shape[0]*remove_percent), replace=False)
    test.drop(test_drop_indices, inplace=True)

513918 of the training dataset is removed


In [19]:
def seq_len_check(row):
    seq_len = 1
    for c in feat_cols:
        seq_len += len(roberta_tok.tokenize(str(row[c]))) + 1    
    return seq_len < (config.max_seq_len - 3)

In [20]:
train.shape

(5192, 4)

In [21]:
if config.seq_len_check:
    train_in_range = train.apply(seq_len_check,axis=1)
    val_in_range = val.apply(seq_len_check,axis=1)
    test_in_range = test.apply(seq_len_check,axis=1)

    print("dropping {} out of {} questions in train".format(sum(~train_in_range),len(train)))
    print("dropping {} out of {} questions in val".format(sum(~val_in_range),len(val)))
    print("dropping {} out of {} questions in test".format(sum(~test_in_range),len(test)))

    train = train[train_in_range]
    val = val[val_in_range]
    test = test[test_in_range]

In [22]:
# randomizing the order of training and val data
train = train.sample(frac=1,random_state = config.seed).reset_index(drop=True)
val = val.sample(frac=1,random_state = config.seed).reset_index(drop=True)


In [23]:
train.head()

,question,passage,entity,label
0,Winner: @placeholder's Marco Tavares (centre) ...,By Mark Wilson for MailOnline For Callum McGre...,Maribor,True
1,When Congress passed a bill banning the use of...,Editor's note: Julian E. Zelizer is a professo...,New Jersey,False
2,"Since @placeholder, each time I go to a game -...",(CNN) -- My prized Penn State sweatshirt sits ...,Sandusky,True
3,"Perez, of @placeholder, agreed that drug traff...",(CNN) -- An isolated Indian tribe in Brazil se...,National Indian Foundation,True
4,Heidi said she wanted a beach wedding original...,"By Katy Winter PUBLISHED: 07:05 EST, 26 Februa...",Heidi,False


In [24]:
train.label.value_counts()

False    2620
True     2572
Name: label, dtype: int64

## Loading from CSVs (alters depending on remove_percent)

In [25]:
train.isna().any()

question    False
passage     False
entity      False
label       False
dtype: bool

In [26]:
train.to_csv(data_path/config.task/"train.csv",index=False)
val.to_csv(data_path/config.task/"val.csv",index=False)
test.to_csv(data_path/config.task/"test.csv",index=False)

In [27]:
train = pd.read_csv(data_path/config.task/"train.csv")
val = pd.read_csv(data_path/config.task/"val.csv")
test = pd.read_csv(data_path/config.task/"test.csv")

In [28]:
train.dropna(inplace=True)
val.dropna(inplace=True)

## Setting up the DataBunch

In [29]:
# Creating a Roberta specific DataBunch class
class RobertaDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training Roberta"
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=64, val_bs:int=None, pad_idx=1,
               pad_first=True, device:torch.device=None, no_check:bool=False, backwards:bool=False, 
               dl_tfms:Optional[Collection[Callable]]=None, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [30]:
class RobertaTextList(TextList):
    _bunch = RobertaDataBunch
    _label_cls = TextList

In [31]:
# loading the tokenizer and vocab processors
processor = get_roberta_processor(tokenizer=fastai_tokenizer, vocab=fastai_roberta_vocab)

# creating our databunch 
data = ItemLists(".", RobertaTextList.from_df(train, ".", cols=feat_cols, processor=processor),
                      RobertaTextList.from_df(val, ".", cols=feat_cols, processor=processor)
                ) \
       .label_from_df(cols=label_cols, label_cls=CategoryList) \
       .add_test(RobertaTextList.from_df(test, ".", cols=feat_cols, processor=processor)) \
       .databunch(bs=config.bs,pad_first=False)

# Building the Model

In [32]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification

# defining our model architecture 
class RobertaForSequenceClassificationModel(nn.Module):
    def __init__(self,num_labels=config.num_labels):
        super(RobertaForSequenceClassificationModel,self).__init__()
        self.num_labels = num_labels
        self.roberta = RobertaForSequenceClassification.from_pretrained(config.roberta_model_name,num_labels= self.num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        outputs = self.roberta(input_ids, token_type_ids, attention_mask)
        logits = outputs[0] 
        return logits

In [33]:
def f_score(input,target,eps:float=1e-9):
    """
    Does not work. Provides faulty results during training.
    """
    n = target.shape[0]
    input = input.argmax(dim=-1).view(n,-1).float()
    target = target.view(n,-1).float()
    TP = (input*target).sum(0)
    prec = TP / (input.sum(0)+eps)
    rec = TP / (target.sum(0)+eps)
    f1 = (2*prec*rec)/(prec+rec+eps)
    return f1

In [34]:
roberta_model = RobertaForSequenceClassificationModel()

learn = Learner(data, roberta_model, metrics=[accuracy])

In [35]:
# run these to find optimal learing rates
# learn.lr_find()
# learn.recorder.plot()

# Getting Untrained Predictions

In [36]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    learn.model.roberta.eval()
    preds = learn.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in data.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    ordered_preds = preds[reverse_sampler, :]
    pred_values = np.argmax(ordered_preds, axis=1)
    return ordered_preds, pred_values

In [37]:
# what % of the train labels are False
(train.label==False).mean()

0.5046224961479199

In [38]:
# what % of the val labels are False
(val.label==False).mean()

0.8339060710194731

In [39]:
# val preds
preds, pred_values = get_preds_as_nparray(DatasetType.Valid)
# accuracy for valid with untrained model
(pred_values == data.valid_ds.y.items).mean()

0.8339060710194731

# Running the model

In [45]:
learn.model.roberta.train() # setting roberta to train as it is in eval mode by default
learn.fit_one_cycle(config.epochs, max_lr=config.max_lr)

# Interpretations

In [46]:
# interpreting training results
interp_train = ClassificationInterpretation.from_learner(learn, ds_type=DatasetType.Train)

In [ ]:
interp_train.plot_confusion_matrix()

In [ ]:
# train accuracy
conf_mat = interp_train.confusion_matrix()
(conf_mat[0,0] + conf_mat[1,1]) / conf_mat.sum()

In [ ]:
# interpreting validation results
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

# Getting Predictions

In [ ]:
def summarize(conf_mat):
    acc = (conf_mat[0,0] + conf_mat[1,1]) / conf_mat.sum()
    rec = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[0,1]) # TP / TP + FN
    prec = conf_mat[1,1] / (conf_mat[1,1] + conf_mat[1,0]) # TP / TP + FP
    f1 = 2*rec*prec/(rec+prec)
    print(f"acc: {acc}, recall: {rec}, precision: {prec}, f1: {f1}")
    

In [ ]:
summarize(interp.confusion_matrix())

In [ ]:
# val preds
pred_values, preds = get_preds_as_nparray(DatasetType.Valid)

In [ ]:
# accuracy for valid 
(preds == data.valid_ds.y.items).mean()

### Test Set Submission Creation

In [47]:
# test preds
test_preds_values, test_preds = get_preds_as_nparray(DatasetType.Test)

In [49]:
test["pred_value"] = list(test_preds_values)
test["pred"] = test_preds

In [50]:
def get_best_idx(x):
    x = np.vstack(x)
    return np.argmax(x[:,-1])

submission = test.groupby("idx").agg({"pred_value":get_best_idx, "entity": lambda x: list(x)})

In [51]:
if "pred_value" in submission.columns:
    submission["label"] = submission.apply(lambda row: row["entity"][row["pred_value"]],1)
    submission.drop(["pred_value","entity"],1,inplace=True)

In [52]:
submission.head()

,label
idx,
13,Boston
15,Lester Holt
16,Union City
23,Ollie Devoto
29,Christians
